# Связь просрочек по кредитным платежам с другими признаками

**В данной работе выяснено: влияет ли семейное положение и количество детей клиента
на факт погашения кредита в срок**

In [1]:
RANDOM_STATE=12345

In [2]:
import pandas as pd

data = pd.read_csv('data.csv')

In [3]:
data.sample(5, random_state=RANDOM_STATE)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10494,0,-777.565227,41,среднее,1,гражданский брак,1,M,сотрудник,0,166520.601913,дополнительное образование
1460,3,-2789.878442,31,среднее,1,женат / замужем,0,M,сотрудник,0,115323.047716,строительство недвижимости
9914,0,NaN,58,высшее,0,женат / замужем,0,M,сотрудник,0,NaN,автомобили
3165,0,329050.424240,59,начальное,3,вдовец / вдова,2,F,пенсионер,0,296160.300798,получение дополнительного образования
3080,0,360739.755029,51,высшее,0,женат / замужем,0,M,пенсионер,0,184394.932593,операции со своей недвижимостью


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

In [5]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце вы обработаете на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.**

In [6]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

In [7]:
(data.days_employed < 0).sum()

15906

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце days_employed. Для реальных данных это нормально. Обработайте значения в этом столбце: замените все отрицательные значения положительными с помощью метода abs().

In [8]:
data['days_employed'] = data['days_employed'].abs()

In [9]:
(data.days_employed < 0).sum()

0

Для каждого типа занятости выведите медианное значение трудового стажа days_employed в днях

In [10]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

In [11]:
data.pivot_table(index='income_type', values='days_employed', aggfunc='median')['days_employed']

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть. Тем более этот столбец не понадобится вам для исследования.

Выведите перечень уникальных значений столбца children.

In [12]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [13]:
data[(data['children'] == -1) | (data['children'] == 20)]['children'].count()

123

В столбце children есть два аномальных значения. Удалите строки, в которых встречаются такие аномальные значения из датафрейма data.

In [14]:
data.shape

(21525, 12)

In [15]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [16]:
data.shape

(21402, 12)

In [17]:
21525-21402

123

In [18]:
data[(data['children'] == -1) | (data['children'] == 20)]['children'].count()

0

In [19]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

Заполните пропуски в столбце days_employed медианными значениями по каждого типа занятости `income_type`

In [20]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [21]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Замените вещественный тип данных в столбце total_income на целочисленный с помощью метода astype().

In [22]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

In [23]:
data.duplicated().sum()

54

In [24]:
data['education'].sample(n=5)

20575    среднее
5464      высшее
4736     среднее
10680    среднее
15942     высшее
Name: education, dtype: object

Обработайте неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведите их к нижнему регистру. Проверьте остальные столбцы.



In [25]:
data['education'] = data['education'].str.lower()

In [26]:
data.duplicated().sum()

71

In [27]:
data = data.drop_duplicates()

### Категоризация данных

На основании диапазонов, указанных ниже, создайте в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используйте собственную функцию с именем `categorize_income()` и метод `apply()`.

In [28]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [29]:
pd.options.mode.copy_on_write = True

In [30]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Выведите на экран перечень уникальных целей взятия кредита из столбца `purpose`.

In [31]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создайте функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category, в который войдут следующие категории:

    'операции с автомобилем',
    'операции с недвижимостью',
    'проведение свадьбы',
    'получение образования'.

Например, если в столбце purpose находится подстрока 'на покупку автомобиля', то в столбце purpose_category должна появиться строка 'операции с автомобилем'.

Используйте собственную функцию с именем categorize_purpose() и метод apply(). Изучите данные в столбце purpose и определите, какие подстроки помогут вам правильно определить категорию.

In [32]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [33]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## Исследовательский анализ данных

### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [34]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21331 entries, 0 to 21524
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   children               21331 non-null  int64  
 1   days_employed          21331 non-null  float64
 2   dob_years              21331 non-null  int64  
 3   education              21331 non-null  object 
 4   education_id           21331 non-null  int64  
 5   family_status          21331 non-null  object 
 6   family_status_id       21331 non-null  int64  
 7   gender                 21331 non-null  object 
 8   income_type            21331 non-null  object 
 9   debt                   21331 non-null  int64  
 10  total_income           21331 non-null  int64  
 11  purpose                21331 non-null  object 
 12  total_income_category  21331 non-null  object 
 13  purpose_category       21331 non-null  object 
dtypes: float64(1), int64(6), object(7)
memory usage: 2.4+ MB


In [35]:
data['debt'].isna().sum()

0

In [36]:
data.shape

(21331, 14)

In [37]:
data = data.dropna(subset=['debt'])

In [38]:
data.shape

(21331, 14)

In [39]:
data['debt'] = data['debt'].astype('int')

In [40]:
data['debt'].value_counts(normalize=True)

debt
0    0.918804
1    0.081196
Name: proportion, dtype: float64

In [41]:
data['children'] = data['children'].astype('int')

In [42]:
data['children'].value_counts()

children
0    14091
1     4808
2     2052
3      330
4       41
5        9
Name: count, dtype: int64

In [43]:
data.groupby('children')['debt'].sum()

children
0    1063
1     444
2     194
3      27
4       4
5       0
Name: debt, dtype: int64

Данных о тех, у кого 4 или 5 детей сравнительно очень мало. Пэтому мы не будем их рассматривать.

In [44]:
(
    round(100.*data.groupby('children')['debt'].sum() / data['children'].value_counts(), 2)
        .sort_values(ascending=False)
        .iloc[1:5]
)

children
2    9.45
1    9.23
3    8.18
0    7.54
dtype: float64

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [45]:
data['family_status'].value_counts()

family_status
женат / замужем          12261
гражданский брак          4134
Не женат / не замужем     2796
в разводе                 1189
вдовец / вдова             951
Name: count, dtype: int64

In [46]:
data.groupby('family_status')['debt'].sum().sort_values(ascending=False)

family_status
женат / замужем          927
гражданский брак         385
Не женат / не замужем    273
в разводе                 84
вдовец / вдова            63
Name: debt, dtype: int64

In [47]:
(
    round(100.*data.groupby('family_status')['debt'].sum() / data['family_status'].value_counts(), 2)
        .sort_values(ascending=False)
)

family_status
Не женат / не замужем    9.76
гражданский брак         9.31
женат / замужем          7.56
в разводе                7.06
вдовец / вдова           6.62
dtype: float64

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [48]:
data['total_income_category'].value_counts()

total_income_category
C    15921
B     5014
D      349
A       25
E       22
Name: count, dtype: int64

In [49]:
data.groupby('total_income_category')['debt'].sum().sort_values(ascending=False)

total_income_category
C    1353
B     354
D      21
A       2
E       2
Name: debt, dtype: int64

Группы A, D, E очень малочислены по стравнеию с B, C. Поэтому мы рассмотрим только B, C, а относительно A предюположим, что они надежные, так как имеют высокий доход. 

In [50]:
(
    round(100.*data.groupby('total_income_category')['debt'].sum() / data['total_income_category'].value_counts(), 2)
        .loc[['B', 'C']]
)

total_income_category
B    7.06
C    8.50
dtype: float64

Получилось, что с ростом доходом - растет надежность.

### Как разные цели кредита влияют на его возврат в срок?

In [51]:
data['purpose_category'].value_counts().sort_values(ascending=False)

purpose_category
операции с недвижимостью    10751
операции с автомобилем       4279
получение образования        3988
проведение свадьбы           2313
Name: count, dtype: int64

In [52]:
data.groupby('purpose_category')['debt'].sum()

purpose_category
операции с автомобилем      400
операции с недвижимостью    780
получение образования       369
проведение свадьбы          183
Name: debt, dtype: int64

In [53]:
(
    round(100.*data.groupby('purpose_category')['debt'].sum() / data['purpose_category'].value_counts(), 2)
        .sort_values(ascending=False)
)

purpose_category
операции с автомобилем      9.35
получение образования       9.25
проведение свадьбы          7.91
операции с недвижимостью    7.26
dtype: float64

Ипотечники оказались самыми надежными заемщиками, а те кто берет кредит на авто - самые не надежные. 

## Вывод

- **Предобработка**
    * В данных заполнены пропуски в стобцах: **`days_employed`** и **`total_income`**, медианными значениями для каждой категории из **`income_type`** отдельно.
    * В столбце **`days_employed`** анамольные отрицательные значения, конвертированы в положительные.
    * В столбце **`total_income`** изменен тип данных перемнных - с вещесвенног на целочисленный.
    * Из таблицы удалено 71 строк-дубликатов.


-  **Категоризация**
    * В столбец **`total_income_category`** записана информация о категорях по доходу, по со следущими обозначениями:

        - 0–30000 — `'E'`;
        - 30001–50000 — `'D'`;
        - 50001–200000 — `'C'`;
        - 200001–1000000 — `'B'`;
        - 1000001 и выше — `'A'`.
    * В столбец **`purpose_category`** записана информация о категориях по цели взатия кредита:
        - `'операции с автомобилем'`,
        - `'операции с недвижимостью'`,
        - `'проведение свадьбы'`,
        - `'получение образования'`


- **Ответы на основные вопросы исследования**
    * Зависимость между количесвом **детей** и возвратом кредита в срок **обнаружена**. Например, те у кого нет детей прцент должников сотавляет **8%**, а у тех у кого 1 и 2 ребенка - **9%** и **10%** соотвесвенно.
    * Зависимость между **семейным положением** и возвратом кредита в срок **обнаружена**. Состоящие и состоявшие в браке допускают просрочки реже чем не женатые / не замужем: примерно **7%** против **10%**.
    * Зависимость между **уровнем дохода** и возвратом кредита в срок **обнаружена**. Более состоятельные клиенты из категории `B` (с доходом от 200 тыс. до 1 миллиона), совершают просрочки реже, чем клиенты из категории `C` (с доходом от 50 тыс. до 200 тыс.) : **7%** против **9%** соответсвенно. 
    * Зависимость между **целью** взятия **кредита** и возвратом его в срок **обнаружена**:
        - `'операции с автомобилем'` - **9%** должников,
        - `'операции с недвижимостью'` - **7%** должников,
        - `'проведение свадьбы'` - **8%** должников,
        - `'получение образования'` - **9%** должников.
    * Протрет самого **надежного клиента** следующий: это **бездентный**, **состящий или состоявший в браке**, с **высоким доходом** (в диапозоне **от 200 тыс. до 1 миллиона**) и берущий **ипотеку**.
    * Потртер самого **ненадежно клиента** следующий: это человек **с детьми**, **не состоявший в браке**, со средним доходом (в диапозоне **от 50 до 200 тыс.**) и берущий **кредит на автомобиль**.